In [1]:
# import random
# import numpy as np
# import pygame
# import sys
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from collections import deque, namedtuple
# import copy

# # Constants
# TILE_SIZE = 24
# GRID_WIDTH = 30
# GRID_HEIGHT = 20
# SCREEN_WIDTH = TILE_SIZE * GRID_WIDTH
# SCREEN_HEIGHT = TILE_SIZE * GRID_HEIGHT
# FPS = 10  # Increased for smoother visualization
# MAX_EPISODES = 10000
# GAMMA = 0.99  # Discount factor
# RENDER = True  # Set to False for faster training

# # Initialize Pygame
# pygame.init()
# screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
# pygame.display.set_caption("Fireboy and Watergirl with RL Agents")
# clock = pygame.time.Clock()

# # Colors
# COLORS = {
#     "0": (120, 66, 18),   # Brown - Empty
#     "1": (160, 82, 45),   # Light Brown - Platform
#     "F": (255, 0, 0),     # Fireboy - Red Heart
#     "W": (0, 0, 255),     # Watergirl - Blue Heart
#     "G": (0, 255, 0),     # Green Agent
#     "RP": (255, 0, 0),    # Red Poison
#     "BP": (0, 0, 255),    # Blue Poison
#     "GP": (0, 255, 0),    # Green Poison
#     "FG": (255, 0, 0),    # Fireboy Goal
#     "WG": (0, 0, 255),    # Watergirl Goal
#     "RB": (255, 0, 0),    # Red Gem
#     "BB": (0, 0, 255),    # Blue Gem
#     "YS": (255, 255, 0),  # Yellow Slide
#     "PS": (128, 0, 128),  # Purple Slide
#     "YB": (255, 255, 0),  # Yellow Button
#     "PB": (128, 0, 128),  # Purple Button
# }

# BASE_COLOR = COLORS["0"]

# # Define action space
# ACTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1), (0, 0)]  # Up, Down, Left, Right, Stay
# NUM_ACTIONS = len(ACTIONS)

# # Experience replay memory for DQN
# Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

# class ReplayMemory:
#     def __init__(self, capacity):
#         self.memory = deque(maxlen=capacity)
    
#     def push(self, state, action, reward, next_state, done):
#         self.memory.append(Experience(state, action, reward, next_state, done))
    
#     def sample(self, batch_size):
#         return random.sample(self.memory, batch_size)
    
#     def __len__(self):
#         return len(self.memory)

# # Neural network for A2C (Actor-Critic)
# class A2CNetwork(nn.Module):
#     def __init__(self, input_dim, action_dim):
#         super(A2CNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 128)
#         self.fc2 = nn.Linear(128, 64)
        
#         # Actor head (policy)
#         self.actor = nn.Linear(64, action_dim)
        
#         # Critic head (value function)
#         self.critic = nn.Linear(64, 1)
        
#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
#         # Actor: Action probabilities
#         action_probs = F.softmax(self.actor(x), dim=-1)
        
#         # Critic: State value
#         state_value = self.critic(x)
        
#         return action_probs, state_value

# # Neural network for DQN
# class DQN(nn.Module):
#     def __init__(self, input_dim, action_dim):
#         super(DQN, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, action_dim)
        
#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         return self.fc3(x)

# # A2C Agent
# class A2CAgent:
#     def __init__(self, input_dim, action_dim, agent_type):
#         self.agent_type = agent_type
#         self.network = A2CNetwork(input_dim, action_dim)
#         self.optimizer = optim.Adam(self.network.parameters(), lr=0.001)
#         self.action_dim = action_dim
        
#     def select_action(self, state, explore=True):
#         state_tensor = torch.FloatTensor(state).unsqueeze(0)
#         action_probs, _ = self.network(state_tensor)
        
#         if explore:
#             # Sample from the probability distribution
#             action_dist = torch.distributions.Categorical(action_probs)
#             action = action_dist.sample().item()
#         else:
#             # Take the most probable action
#             action = torch.argmax(action_probs).item()
        
#         return action
    
#     def update(self, state, action, reward, next_state, done):
#         # Convert to tensors
#         state_tensor = torch.FloatTensor(state).unsqueeze(0)
#         next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)
#         reward_tensor = torch.FloatTensor([reward])
#         done_tensor = torch.FloatTensor([done])
        
#         # Get current policy and value
#         action_probs, state_value = self.network(state_tensor)
        
#         # Calculate next state value
#         _, next_state_value = self.network(next_state_tensor)
#         next_state_value = next_state_value.detach()
        
#         # Calculate target and advantage
#         target = reward_tensor + GAMMA * next_state_value * (1 - done_tensor)
#         advantage = target - state_value
        
#         # Calculate losses
#         action_dist = torch.distributions.Categorical(action_probs)
#         action_log_probs = action_dist.log_prob(torch.tensor(action))
        
#         # Actor loss (policy gradient)
#         actor_loss = -action_log_probs * advantage.detach()
        
#         # Critic loss (value function)
#         critic_loss = F.mse_loss(state_value, target.detach())
        
#         # Entropy to encourage exploration
#         entropy = -torch.sum(action_probs * torch.log(action_probs + 1e-8))
        
#         # Total loss
#         total_loss = actor_loss + 0.5 * critic_loss - 0.01 * entropy
        
#         # Update network
#         self.optimizer.zero_grad()
#         total_loss.backward()
#         self.optimizer.step()

# # DQN Agent
# class DQNAgent:
#     def __init__(self, input_dim, action_dim):
#         self.input_dim = input_dim
#         self.action_dim = action_dim
#         self.policy_net = DQN(input_dim, action_dim)
#         self.target_net = DQN(input_dim, action_dim)
#         self.target_net.load_state_dict(self.policy_net.state_dict())
#         self.target_update_freq = 100
#         self.optimizer = optim.Adam(self.policy_net.parameters(), lr=0.001)
#         self.memory = ReplayMemory(10000)
#         self.batch_size = 64
#         self.epsilon = 1.0
#         self.epsilon_min = 0.1
#         self.epsilon_decay = 0.995
#         self.gamma = GAMMA
#         self.steps_done = 0
        
#     def select_action(self, state, explore=True):
#         if explore and random.random() < self.epsilon:
#             return random.randrange(self.action_dim)
#         else:
#             with torch.no_grad():
#                 state_tensor = torch.FloatTensor(state).unsqueeze(0)
#                 return self.policy_net(state_tensor).max(1)[1].item()
    
#     def update_epsilon(self):
#         self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
#     def update_target_network(self):
#         if self.steps_done % self.target_update_freq == 0:
#             self.target_net.load_state_dict(self.policy_net.state_dict())
    
#     def push_to_memory(self, state, action, reward, next_state, done):
#         self.memory.push(state, action, reward, next_state, done)
#         self.steps_done += 1
    
#     def learn(self):
#         if len(self.memory) < self.batch_size:
#             return
        
#         experiences = self.memory.sample(self.batch_size)
#         batch = Experience(*zip(*experiences))
        
#         state_batch = torch.FloatTensor(batch.state)
#         action_batch = torch.LongTensor(batch.action).unsqueeze(1)
#         reward_batch = torch.FloatTensor(batch.reward)
#         next_state_batch = torch.FloatTensor(batch.next_state)
#         done_batch = torch.FloatTensor(batch.done)
        
#         # Compute Q(s_t, a) - the model computes Q(s_t), then we select the columns of actions taken
#         state_action_values = self.policy_net(state_batch).gather(1, action_batch)
        
#         # Compute V(s_{t+1}) for all next states
#         next_state_values = torch.zeros(self.batch_size)
#         with torch.no_grad():
#             next_state_values = self.target_net(next_state_batch).max(1)[0]
        
#         # Compute the expected Q values
#         expected_state_action_values = reward_batch + (self.gamma * next_state_values * (1 - done_batch))
        
#         # Compute Huber loss
#         loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
        
#         # Optimize the model
#         self.optimizer.zero_grad()
#         loss.backward()
#         for param in self.policy_net.parameters():
#             param.grad.data.clamp_(-1, 1)
#         self.optimizer.step()

# # Game Environment
# class GameEnvironment:
#     def __init__(self):
#         self.reset()
        
#         # Define state size: flattened grid plus agent positions
#         self.state_size = (GRID_WIDTH * GRID_HEIGHT) + 6  # Grid + agent positions (y,x for each agent)
#         self.action_size = NUM_ACTIONS
        
#         # Initialize agents
#         self.a2c_agent_fire = A2CAgent(self.state_size, self.action_size, "F")
#         self.a2c_agent_water = A2CAgent(self.state_size, self.action_size, "W")
#         self.dqn_agent_green = DQNAgent(self.state_size, self.action_size)
        
#         # Track buttons pressed
#         self.yellow_buttons_pressed = 0
#         self.purple_buttons_pressed = 0
#         self.total_buttons = 0  # Will be counted during reset
        
#         # Track collaboration
#         self.fire_water_cooperation = False
        
#         # Track goals reached
#         self.fire_reached_goal = False
#         self.water_reached_goal = False
        
#         # Track gems collected
#         self.fire_gems = 0
#         self.water_gems = 0
#         self.total_fire_gems = 0
#         self.total_water_gems = 0
    
#     def reset(self):
#         # Create grid
#         self.grid = [["0" for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        
#         # Initial agent positions
#         self.agents = {
#             "F": [18, 1],
#             "W": [15, 1],
#             "G": [2, 19]
#         }
        
#         # Store original grid state
#         self.original_grid = None
        
#         # Place agents on grid
#         for agent, pos in self.agents.items():
#             self.grid[pos[0]][pos[1]] = agent
        
#         # Poison tiles
#         for x in range(10, 13): self.grid[18][x] = "RP"
#         for x in range(17, 20): self.grid[18][x] = "BP"
#         for x in range(18, 21): self.grid[14][x] = "GP"
        
#         # Gems
#         self.grid[17][11] = self.grid[8][10] = "RB"
#         self.grid[17][18] = self.grid[10][20] = "BB"
        
#         # Count total gems
#         self.total_fire_gems = sum(row.count("RB") for row in self.grid)
#         self.total_water_gems = sum(row.count("BB") for row in self.grid)
        
#         # Goals
#         self.grid[2][26] = "FG"
#         self.grid[2][28] = "WG"
        
#         # Slides
#         for x in range(1, 4): self.grid[9][x] = "YS"
#         for x in range(26, 29): self.grid[6][x] = "PS"
        
#         # Buttons
#         self.grid[8][6] = self.grid[12][9] = "YB"
#         self.grid[8][14] = self.grid[5][22] = "PB"
        
#         # Count total buttons
#         self.total_buttons = sum(row.count("YB") for row in self.grid) + sum(row.count("PB") for row in self.grid)
        
#         # Platforms (walls)
#         for i in range(30): self.grid[0][i] = self.grid[19][i] = "1"
#         for i in range(20): self.grid[i][0] = self.grid[i][29] = "1"
#         self.grid[1][1] = self.grid[1][2] = "1"
#         for i in range(4, 29): self.grid[3][i] = "1"
#         self.grid[4][1] = self.grid[4][2] = self.grid[5][1] = self.grid[5][2] = "1"
#         for i in range(1, 26): self.grid[6][i] = "1"
#         self.grid[7][18] = self.grid[7][19] = self.grid[7][20] = "1"
#         self.grid[8][18] = self.grid[8][19] = self.grid[8][20] = "1"
#         for i in range(4, 16): self.grid[9][i] = "1"
#         for i in range(17, 22): self.grid[11][i] = "1"
#         for i in range(22, 29): self.grid[10][i] = "1"
#         self.grid[10][16] = "1"
#         self.grid[11][26] = self.grid[11][27] = self.grid[11][28] = "1"
#         self.grid[12][26] = self.grid[12][27] = self.grid[12][28] = "1"
#         for i in range(1, 13): self.grid[13][i] = "1"
#         self.grid[14][13] = "1"
#         for i in range(14, 27): self.grid[15][i] = "1"
#         for i in range(1, 7): self.grid[16][i] = "1"
#         self.grid[17][27] = self.grid[17][28] = self.grid[18][27] = self.grid[18][28] = "1"
        
#         # Store original grid after setup (for resetting after collisions)
#         self.original_grid = [row[:] for row in self.grid]
        
#         # Reset game state
#         self.yellow_buttons_pressed = 0
#         self.purple_buttons_pressed = 0
#         self.fire_water_cooperation = False
#         self.fire_reached_goal = False
#         self.water_reached_goal = False
#         self.fire_gems = 0
#         self.water_gems = 0
#         self.done = False
        
#         return self.get_state()
    
#     def get_state(self):
#         # Flattened grid representation (one-hot encoded)
#         # For simplicity, we'll use a simpler representation here
#         grid_flat = []
#         for row in self.grid:
#             for cell in row:
#                 if cell == "0":
#                     grid_flat.append(0)  # Empty
#                 elif cell == "1":
#                     grid_flat.append(1)  # Wall
#                 elif cell == "F":
#                     grid_flat.append(2)  # Fireboy
#                 elif cell == "W":
#                     grid_flat.append(3)  # Watergirl
#                 elif cell == "G":
#                     grid_flat.append(4)  # Green agent
#                 elif cell == "RP":
#                     grid_flat.append(5)  # Red poison
#                 elif cell == "BP":
#                     grid_flat.append(6)  # Blue poison
#                 elif cell == "GP":
#                     grid_flat.append(7)  # Green poison
#                 elif cell == "FG":
#                     grid_flat.append(8)  # Fireboy goal
#                 elif cell == "WG":
#                     grid_flat.append(9)  # Watergirl goal
#                 elif cell == "RB":
#                     grid_flat.append(10)  # Red gem
#                 elif cell == "BB":
#                     grid_flat.append(11)  # Blue gem
#                 elif cell == "YS":
#                     grid_flat.append(12)  # Yellow slide
#                 elif cell == "PS":
#                     grid_flat.append(13)  # Purple slide
#                 elif cell == "YB":
#                     grid_flat.append(14)  # Yellow button
#                 elif cell == "PB":
#                     grid_flat.append(15)  # Purple button
#                 else:
#                     grid_flat.append(0)  # Default to empty
                    
#         # Add agent positions
#         for agent, pos in self.agents.items():
#             grid_flat.extend(pos)
            
#         return grid_flat
    
#     def is_valid_move(self, agent, y, x):
#         # Check bounds
#         if not (0 <= x < GRID_WIDTH and 0 <= y < GRID_HEIGHT):
#             return False
        
#         # Check if the cell is a wall
#         if self.grid[y][x] == "1":
#             return False
        
#         # Check if the cell contains another agent
#         if self.grid[y][x] in ["F", "W", "G"] and self.grid[y][x] != agent:
#             return False
            
#         return True

#     def check_special_interaction(self, agent, y, x):
#         cell = self.grid[y][x]
#         rewards = {"F": 0, "W": 0, "G": 0}
#         game_over = False
        
#         # Poison interaction - preserve poison tiles
#         if agent == "F" and cell == "BP":
#             rewards["F"] = -10  # Fireboy dies in red poison
#             rewards["W"] = -5
#             rewards["G"] = 5    # Green agent gets rewarded when opponents fail
#             game_over = True
#         elif agent == "F" and cell == "RP":
#             rewards["F"] = -5   # Fireboy gets penalty for red poison
#             rewards["W"] = 0
#             rewards["G"] = 2    # Small reward for green agent
#             self.fire_poison = True
#         elif agent == "W" and cell == "RP":
#             rewards["W"] = -10  # Watergirl dies in blue poison
#             rewards["F"] = -5
#             rewards["G"] = 5    # Green agent gets rewarded when opponents fail
#             game_over = True
#         elif agent == "W" and cell == "BP":
#             rewards["W"] = -5   # Watergirl gets penalty for blue poison
#             rewards["F"] = 0
#             rewards["G"] = 2    # Small reward for green agent
#             self.water_poison = True
#         elif agent in ["F", "W"] and cell == "GP":
#             rewards[agent] = -10  # Both die in green poison
#             rewards["G"] = -10    # Green agent also penalized for green poison
#             game_over = True
#         elif agent == "G" and cell in ["RP", "BP", "GP"]:
#             rewards["G"] = -5     # Green agent penalized for touching any poison
#             game_over = False
        
#         # Gem collection
#         if agent == "F" and cell == "RB":
#             rewards["F"] = 5    # Fireboy collects red gem
#             rewards["W"] = 1
#             rewards["G"] = -2   # Green agent penalized when opponents succeed
#             self.fire_gems += 1
#             self.grid[y][x] = "0"  # Remove the gem (gems should be collected)
#         elif agent == "W" and cell == "BB":
#             rewards["W"] = 5    # Watergirl collects blue gem
#             rewards["F"] = 1
#             rewards["G"] = -2   # Green agent penalized when opponents succeed
#             self.water_gems += 1
#             self.grid[y][x] = "0"  # Remove the gem (gems should be collected)
#         elif agent == "G" and cell in ["RB", "BB"]:
#             rewards["G"] = 5    # Green agent collects gems
#             rewards["F"] = -1   # Small penalty for F&W when green collects gems
#             rewards["W"] = -1
#             self.grid[y][x] = "0"  # Remove the gem

#         # Wrong gem collection penalties
#         elif agent == "F" and cell == "BB":
#             rewards["F"] = -2    # Penalty for Fireboy trying to collect blue gem
#             rewards["W"] = -1    # Small penalty for Watergirl
#             rewards["G"] = 2     # Green agent gets rewarded when opponents fail
#             self.water_gem = True
#         elif agent == "W" and cell == "RB":
#             rewards["W"] = -2    # Penalty for Watergirl trying to collect red gem
#             rewards["F"] = -1    # Small penalty for Fireboy
#             rewards["G"] = 2     # Green agent gets rewarded when opponents fail
#             self.fire_gem = True
        
#         # Goal reached - goals remain in place
#         if agent == "F" and cell == "FG":
#             rewards["F"] = 10    # Fireboy reaches goal
#             rewards["W"] = 3
#             rewards["G"] = -10   # Green agent gets penalty
#             self.fire_reached_goal = True
#             # Goal remains in place
#         elif agent == "W" and cell == "WG":
#             rewards["W"] = 10    # Watergirl reaches goal
#             rewards["F"] = 3
#             rewards["G"] = -10   # Green agent gets penalty
#             self.water_reached_goal = True
#             # Goal remains in place
#         elif agent == "G" and cell in ["FG", "WG"]:
#             rewards["G"] = -5    # Green agent penalized for touching goals
#             rewards["F"] = 0
#             rewards["W"] = 0
            
#         # Button pressing - buttons remain in place
#         if agent in ["F", "W", "G"] and cell == "YB":
#             # Yellow button activation
#             if agent == "G":
#                 rewards["G"] = 3
#                 rewards["F"] = -1
#                 rewards["W"] = -1
#             else:
#                 # Check if both F&W agents are near each other (cooperation)
#                 f_pos = self.agents["F"]
#                 w_pos = self.agents["W"]
#                 if abs(f_pos[0] - w_pos[0]) + abs(f_pos[1] - w_pos[1]) <= 3:  # Manhattan distance <= 3
#                     rewards["F"] = 8
#                     rewards["W"] = 8
#                     rewards["G"] = -5    # Greater penalty for green agent when F&W cooperate
#                     self.fire_water_cooperation = True
#                 else:
#                     rewards["F"] = -2 if agent == "F" else 0
#                     rewards["W"] = -2 if agent == "W" else 0
#                     rewards["G"] = 2     # Green agent rewarded when F&W fail to cooperate
            
#             # Track which agent is on the button and open/close slides accordingly
#             self.yellow_button_pressed_by = agent
#             self.yellow_buttons_pressed += 1
#             # Open yellow slides - will be handled in step method
                
#         if agent in ["F", "W", "G"] and cell == "PB":
#             # Purple button activation
#             if agent == "G":
#                 rewards["G"] = 3
#                 rewards["F"] = -1
#                 rewards["W"] = -1
#             else:
#                 # Check if both F&W agents are near each other (cooperation)
#                 f_pos = self.agents["F"]
#                 w_pos = self.agents["W"]
#                 if abs(f_pos[0] - w_pos[0]) + abs(f_pos[1] - w_pos[1]) <= 3:  # Manhattan distance <= 3
#                     rewards["F"] = 8
#                     rewards["W"] = 8
#                     rewards["G"] = -5    # Greater penalty for green agent when F&W cooperate
#                     self.fire_water_cooperation = True
#                 else:
#                     rewards["F"] = -2 if agent == "F" else 0
#                     rewards["W"] = -2 if agent == "W" else 0
#                     rewards["G"] = 2     # Green agent rewarded when F&W fail to cooperate
                    
#             # Track which agent is on the button and open/close slides accordingly
#             self.purple_button_pressed_by = agent
#             self.purple_buttons_pressed += 1
#             # Open purple slides - will be handled in step method

#         # Additional rewards for proximity to encourage cooperation
#         f_pos = self.agents["F"]
#         w_pos = self.agents["W"]
#         proximity = abs(f_pos[0] - w_pos[0]) + abs(f_pos[1] - w_pos[1])
#         if proximity <= 2 and agent in ["F", "W"]:
#             rewards["F"] += 0.2
#             rewards["W"] += 0.2
#             rewards["G"] -= 0.1  # Small penalty for green when F&W are cooperating
        
#         return rewards, game_over
    
#     def step(self, fire_action, water_action, green_action):
#         if self.done:
#             return self.get_state(), {"F": 0, "W": 0, "G": 0}, self.done, {}
            
#         rewards = {"F": 0, "W": 0, "G": 0}
#         game_over = False
        
#         # Reset button states at the beginning of each step
#         previous_yellow_pressed = getattr(self, "yellow_button_pressed_by", None)
#         previous_purple_pressed = getattr(self, "purple_button_pressed_by", None)
#         self.yellow_button_pressed_by = None
#         self.purple_button_pressed_by = None
        
#         # Get grid dimensions
#         height = len(self.grid)
#         width = len(self.grid[0]) if height > 0 else 0
        
#         # First pass: Handle and track slide states based on button states
#         # Replace yellow slides with empty space when yellow button is pressed
#         if previous_yellow_pressed is not None:
#             for y in range(height):
#                 for x in range(width):
#                     if self.grid[y][x] == "YS":
#                         self.grid[y][x] = "0"  # Open yellow slides
#                         if not hasattr(self, "yellow_slide_positions"):
#                             self.yellow_slide_positions = []
#                         self.yellow_slide_positions.append((y, x))  # Track positions for later
        
#         # Replace purple slides with empty space when purple button is pressed
#         if previous_purple_pressed is not None:
#             for y in range(height):
#                 for x in range(width):
#                     if self.grid[y][x] == "PS":
#                         self.grid[y][x] = "0"  # Open purple slides
#                         if not hasattr(self, "purple_slide_positions"):
#                             self.purple_slide_positions = []
#                         self.purple_slide_positions.append((y, x))  # Track positions for later
        
#         # Perform actions
#         agents_actions = {
#             "F": fire_action,
#             "W": water_action,
#             "G": green_action
#         }
        
#         # Process moves for each agent
#         for agent, action in agents_actions.items():
#             y, x = self.agents[agent]
#             dy, dx = ACTIONS[action]
#             ny, nx = y + dy, x + dx
            
#             if self.is_valid_move(agent, ny, nx):
#                 # Check for special interactions
#                 interaction_rewards, is_game_over = self.check_special_interaction(agent, ny, nx)
                
#                 for a, r in interaction_rewards.items():
#                     rewards[a] += r
                    
#                 if is_game_over:
#                     game_over = True
#                     # We don't break here to allow other agents to move even if one dies
                
#                 # Move the agent without overwriting special tiles
#                 current_cell = self.grid[y][x]
#                 target_cell = self.grid[ny][nx]
                
#                 # Don't overwrite special tiles when moving away
#                 if current_cell == agent:  # Only clear if the agent is there (not on a special tile)
#                     self.grid[y][x] = "0"  # Clear previous position
                
#                 # For special tiles, we want to remember what they are
#                 # Save the target cell before overwriting
#                 if target_cell in ["YB", "PB", "FG", "WG", "RP", "BP", "GP"]:
#                     # Store the original cell value in a dictionary
#                     if not hasattr(self, "original_cells"):
#                         self.original_cells = {}
#                     self.original_cells[(ny, nx)] = target_cell
                
#                 # Move agent to new position
#                 self.grid[ny][nx] = agent  # Set new position
#                 self.agents[agent] = [ny, nx]  # Update agent position
        
#         # Third pass: Close slides if buttons are no longer pressed
#         # Restore yellow slides if button is not pressed
#         if previous_yellow_pressed is not None and self.yellow_button_pressed_by is None:
#             for y, x in getattr(self, "yellow_slide_positions", []):
#                 # Make sure indices are within bounds
#                 if 0 <= y < len(self.grid) and 0 <= x < len(self.grid[0]):
#                     self.grid[y][x] = "YS"  # Close yellow slides
#             self.yellow_slide_positions = []  # Reset tracking
        
#         # Restore purple slides if button is not pressed
#         if previous_purple_pressed is not None and self.purple_button_pressed_by is None:
#             for y, x in getattr(self, "purple_slide_positions", []):
#                 # Make sure indices are within bounds
#                 if 0 <= y < len(self.grid) and 0 <= x < len(self.grid[0]):
#                     self.grid[y][x] = "PS"  # Close purple slides
#             self.purple_slide_positions = []  # Reset tracking
        
#         # Check if both reached their goals
#         if self.fire_reached_goal and self.water_reached_goal:
#             rewards["F"] += 20
#             rewards["W"] += 20
#             rewards["G"] -= 20  # Double penalty for green agent
#             game_over = True
            
#         # Restore special tiles if agents have moved off them
#         for agent in ["F", "W", "G"]:
#             y, x = self.agents[agent]
#             pos_key = (y, x)
#             if hasattr(self, "original_cells") and pos_key in self.original_cells:
#                 # Agent is on a special tile, nothing to restore
#                 pass
#             else:
#                 # Check if the previous position was a special tile that needs to be restored
#                 prev_pos = getattr(self, f"{agent.lower()}_prev_pos", None)
#                 if prev_pos:
#                     prev_y, prev_x = prev_pos
#                     prev_pos_key = (prev_y, prev_x)
#                     if hasattr(self, "original_cells") and prev_pos_key in self.original_cells:
#                         if self.grid[prev_y][prev_x] == "0":  # Only restore if the cell is empty
#                             self.grid[prev_y][prev_x] = self.original_cells[prev_pos_key]
            
#             # Update previous position for next step
#             setattr(self, f"{agent.lower()}_prev_pos", (y, x))
            
#         self.done = game_over
        
#         return self.get_state(), rewards, self.done, {}
    
#     def render(self):
#         if not RENDER:
#             return
            
#         screen.fill((0, 0, 0))
        
#         for y in range(GRID_HEIGHT):
#             for x in range(GRID_WIDTH):
#                 tile = self.grid[y][x]
#                 color = COLORS.get(tile, BASE_COLOR)

#                 # Agent hearts
#                 if tile in ["F", "W", "G"]:
#                     pygame.draw.rect(screen, color, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
#                     pygame.draw.polygon(screen, color, [
#                         (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + 6),
#                         (x*TILE_SIZE + 6, y*TILE_SIZE + TILE_SIZE//2),
#                         (x*TILE_SIZE + TILE_SIZE - 6, y*TILE_SIZE + TILE_SIZE//2),
#                     ])
#                     pygame.draw.circle(screen, color, (x*TILE_SIZE + TILE_SIZE//3, y*TILE_SIZE + TILE_SIZE//3), 5)
#                     pygame.draw.circle(screen, color, (x*TILE_SIZE + 2*TILE_SIZE//3, y*TILE_SIZE + TILE_SIZE//3), 5)

#                 # Gems
#                 elif tile in ["RB", "BB"]:
#                     pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
#                     pygame.draw.polygon(screen, color, [
#                         (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE),
#                         (x*TILE_SIZE + TILE_SIZE, y*TILE_SIZE + TILE_SIZE//2),
#                         (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + TILE_SIZE),
#                         (x*TILE_SIZE, y*TILE_SIZE + TILE_SIZE//2)
#                     ])

#                 # Buttons
#                 elif tile in ["YB", "PB"]:
#                     pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
#                     pygame.draw.circle(screen, color, (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + TILE_SIZE//2), TILE_SIZE//4)

#                 # Goals
#                 elif tile in ["FG", "WG"]:
#                     pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
#                     pygame.draw.polygon(screen, color, [
#                         (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE),
#                         (x*TILE_SIZE, y*TILE_SIZE + TILE_SIZE),
#                         (x*TILE_SIZE + TILE_SIZE, y*TILE_SIZE + TILE_SIZE)
#                     ])

#                 else:
#                     pygame.draw.rect(screen, color, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))

#                 pygame.draw.rect(screen, (160, 82, 45), (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)
                
#         # Display stats
#         font = pygame.font.SysFont('Arial', 14)
        
#         # # Fire stats
#         # fire_text = f"Fire: G:{self.fire_gems}/{self.total_fire_gems} Goal:{self.fire_reached_goal}"
#         # fire_surf = font.render(fire_text, True, COLORS["F"])
#         # screen.blit(fire_surf, (10, 10))
        
#         # # Water stats
#         # water_text = f"Water: G:{self.water_gems}/{self.total_water_gems} Goal:{self.water_reached_goal}"
#         # water_surf = font.render(water_text, True, COLORS["W"])
#         # screen.blit(water_surf, (10, 30))
        
#         # # Button stats
#         # button_text = f"Buttons: Y:{self.yellow_buttons_pressed} P:{self.purple_buttons_pressed} Coop:{self.fire_water_cooperation}"
#         # button_surf = font.render(button_text, True, (255, 255, 255))
#         # screen.blit(button_surf, (10, 50))
        
#         pygame.display.flip()
#         clock.tick(FPS)

# def train():
#     env = GameEnvironment()
    
#     # Training parameters
#     episodes = MAX_EPISODES
#     max_steps = 1000
    
#     for episode in range(episodes):
#         state = env.reset()
#         total_rewards = {"F": 0, "W": 0, "G": 0}
        
#         for step in range(max_steps):
#             # Handle Pygame events
#             for event in pygame.event.get():
#                 if event.type == pygame.QUIT:
#                     pygame.quit()
#                     sys.exit()
            
#             # Get actions from agents
#             fire_action = env.a2c_agent_fire.select_action(state)
#             water_action = env.a2c_agent_water.select_action(state)
#             green_action = env.dqn_agent_green.select_action(state)

#             # ADDED: Encourage cooperation by occasionally forcing similar actions
#             if random.random() < 0.1:  # 10% chance for cooperative action
#                 if random.random() < 0.5:
#                     water_action = fire_action  # Watergirl follows Fireboy
#                 else:
#                     fire_action = water_action  # Fireboy follows Watergirl
            
#             # Take step in environment
#             next_state, rewards, done, _ = env.step(fire_action, water_action, green_action)
            
#             # Update agents
#             env.a2c_agent_fire.update(state, fire_action, rewards["F"], next_state, done)
#             env.a2c_agent_water.update(state, water_action, rewards["W"], next_state, done)
            
#             # For DQN agent, we push to memory and learn periodically
#             env.dqn_agent_green.push_to_memory(state, green_action, rewards["G"], next_state, done)
#             env.dqn_agent_green.learn()
            
#             # Accumulate rewards
#             for agent in ["F", "W", "G"]:
#                 total_rewards[agent] += rewards[agent]
            
#             # Render the environment
#             env.render()
            
#             # Update state
#             state = next_state
            
#             # Check if episode is done
#             if done:
#                 break
        
#         # Update DQN epsilon and target network
#         env.dqn_agent_green.update_epsilon()
#         env.dqn_agent_green.update_target_network()
        
#         # Print episode statistics
#         print(f"Episode {episode}/{episodes} - Steps: {step}")
#         print(f"Rewards - Fire: {total_rewards['F']:.2f}, Water: {total_rewards['W']:.2f}, Green: {total_rewards['G']:.2f}")
#         print(f"Fire Goal: {env.fire_reached_goal}, Water Goal: {env.water_reached_goal}")
#         print(f"Gems - Fire: {env.fire_gems}/{env.total_fire_gems}, Water: {env.water_gems}/{env.total_water_gems}")
#         print(f"Buttons - Yellow: {env.yellow_buttons_pressed}, Purple: {env.purple_buttons_pressed}")
#         print(f"Cooperation: {env.fire_water_cooperation}")
#         print("-" * 50)
        
#         # Save model checkpoints periodically
#         if episode % 100 == 0:
#             torch.save(env.a2c_agent_fire.network.state_dict(), f"fire_agent_episode_{episode}.pth")
#             torch.save(env.a2c_agent_water.network.state_dict(), f"water_agent_episode_{episode}.pth")
#             torch.save(env.dqn_agent_green.policy_net.state_dict(), f"green_agent_episode_{episode}.pth")

# def test(num_episodes=10):
#     env = GameEnvironment()
    
#     # Load trained models
#     env.a2c_agent_fire.network.load_state_dict(torch.load("fire_agent_episode_9900.pth"))
#     env.a2c_agent_water.network.load_state_dict(torch.load("water_agent_episode_9900.pth"))
#     env.dqn_agent_green.policy_net.load_state_dict(torch.load("green_agent_episode_9900.pth"))
    
#     for episode in range(num_episodes):
#         state = env.reset()
#         done = False
#         step = 0
        
#         while not done and step < 1000:
#             # Handle Pygame events
#             for event in pygame.event.get():
#                 if event.type == pygame.QUIT:
#                     pygame.quit()
#                     sys.exit()
            
#             # Get actions from agents - no exploration during testing
#             fire_action = env.a2c_agent_fire.select_action(state, explore=False)
#             water_action = env.a2c_agent_water.select_action(state, explore=False)
#             green_action = env.dqn_agent_green.select_action(state, explore=False)
            
#             # Take step in environment
#             state, rewards, done, _ = env.step(fire_action, water_action, green_action)
            
#             # Render the environment
#             env.render()
            
#             step += 1
            
#             # Slow down for visualization
#             pygame.time.delay(100)
        
#         print(f"Test Episode {episode+1} completed in {step} steps")
#         pygame.time.delay(1000)  # Pause between episodes

# if __name__ == "__main__":
#     # Train the agents
#     train()
    
#     # Test the trained agents
#     test()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Episode 0/10000 - Steps: 42
Rewards - Fire: -9.60, Water: -4.60, Green: 4.80
Fire Goal: False, Water Goal: False
Gems - Fire: 0/2, Water: 0/2
Buttons - Yellow: 0, Purple: 0
Cooperation: False
--------------------------------------------------
Episode 1/10000 - Steps: 29
Rewards - Fire: -10.00, Water: -4.00, Green: 5.00
Fire Goal: False, Water Goal: False
Gems - Fire: 1/2, Water: 0/2
Buttons - Yellow: 0, Purple: 0
Cooperation: False
--------------------------------------------------
Episode 2/10000 - Steps: 17
Rewards - Fire: -25.00, Water: -5.00, Green: 11.00
Fire Goal: False, Water Goal: False
Gems - Fire: 0/2, Water: 0/2
Buttons - Yellow: 0, Purple: 0
Cooperation: False
--------------------------------------------------
Episode 3/10000 - Steps: 15
Rewards - Fire: -25.00, Water: -5.00, Green: 11.00
Fire Goal: False, Water Goal: False
Gems - Fire: 0/2, Water: 0/2
Buttons - Y

SystemExit: 

C:\Users\Srinu\anaconda3\envs\myenv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import random
import numpy as np
import pygame
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque, namedtuple
import copy

# Constants
TILE_SIZE = 24
GRID_WIDTH = 30
GRID_HEIGHT = 20
SCREEN_WIDTH = TILE_SIZE * GRID_WIDTH
SCREEN_HEIGHT = TILE_SIZE * GRID_HEIGHT
FPS = 10  # Increased for smoother visualization
MAX_EPISODES = 10000
GAMMA = 0.99  # Discount factor
RENDER = True  # Set to False for faster training

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Fireboy and Watergirl with RL Agents")
clock = pygame.time.Clock()

# Colors
COLORS = {
    "0": (120, 66, 18),   # Brown - Empty
    "1": (160, 82, 45),   # Light Brown - Platform
    "F": (255, 0, 0),     # Fireboy - Red Heart
    "W": (0, 0, 255),     # Watergirl - Blue Heart
    "G": (0, 255, 0),     # Green Agent
    "RP": (255, 0, 0),    # Red Poison
    "BP": (0, 0, 255),    # Blue Poison
    "GP": (0, 255, 0),    # Green Poison
    "FG": (255, 0, 0),    # Fireboy Goal
    "WG": (0, 0, 255),    # Watergirl Goal
    "RB": (255, 0, 0),    # Red Gem
    "BB": (0, 0, 255),    # Blue Gem
    "YS": (255, 255, 0),  # Yellow Slide
    "PS": (128, 0, 128),  # Purple Slide
    "YB": (255, 255, 0),  # Yellow Button
    "PB": (128, 0, 128),  # Purple Button
}

BASE_COLOR = COLORS["0"]

# Define action space
ACTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1), (0, 0)]  # Up, Down, Left, Right, Stay
NUM_ACTIONS = len(ACTIONS)

# Experience replay memory for DQN
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.memory.append(Experience(state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

# Modified A2C network to handle both Fireboy and Watergirl jointly
class SharedA2CNetwork(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(SharedA2CNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)  # Increased network capacity
        self.fc2 = nn.Linear(256, 128)
        
        # Two actor heads, one for each agent
        self.actor_fire = nn.Linear(128, action_dim)
        self.actor_water = nn.Linear(128, action_dim)
        
        # Two critic heads, one for each agent
        self.critic_fire = nn.Linear(128, 1)
        self.critic_water = nn.Linear(128, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Actor: Action probabilities for each agent
        action_probs_fire = F.softmax(self.actor_fire(x), dim=-1)
        action_probs_water = F.softmax(self.actor_water(x), dim=-1)
        
        # Critic: State values for each agent
        state_value_fire = self.critic_fire(x)
        state_value_water = self.critic_water(x)
        
        return action_probs_fire, action_probs_water, state_value_fire, state_value_water

# Neural network for DQN (unchanged)
class DQN(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Shared A2C Agent for both Fireboy and Watergirl
class SharedA2CAgent:
    def __init__(self, input_dim, action_dim):
        self.network = SharedA2CNetwork(input_dim, action_dim)
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.001)
        self.action_dim = action_dim
        
    def select_action(self, state, agent_type, explore=True):
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        action_probs_fire, action_probs_water, _, _ = self.network(state_tensor)
        
        # Select action based on agent type
        action_probs = action_probs_fire if agent_type == "F" else action_probs_water
        
        if explore:
            # Sample from the probability distribution
            action_dist = torch.distributions.Categorical(action_probs)
            action = action_dist.sample().item()
        else:
            # Take the most probable action
            action = torch.argmax(action_probs).item()
        
        return action
    
    def update(self, state, fire_action, water_action, fire_reward, water_reward, next_state, done):
        position_fire = torch.tensor(state[-6:-4], dtype=torch.float32)
        position_water = torch.tensor(state[-4:-2], dtype=torch.float32)
        distance = torch.norm(position_fire - position_water)

        # proximity_loss = torch.where(distance < 3.0, action_similarity, torch.tensor(0.0, dtype=torch.float32))
        
        # Convert to tensors
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)
        fire_reward_tensor = torch.FloatTensor([fire_reward])
        water_reward_tensor = torch.FloatTensor([water_reward])
        done_tensor = torch.FloatTensor([done])
        
        # Get current policy and values
        action_probs_fire, action_probs_water, state_value_fire, state_value_water = self.network(state_tensor)
        
        # Calculate next state values
        _, _, next_state_value_fire, next_state_value_water = self.network(next_state_tensor)
        next_state_value_fire = next_state_value_fire.detach()
        next_state_value_water = next_state_value_water.detach()
        
        # Calculate targets and advantages for both agents
        fire_target = fire_reward_tensor + GAMMA * next_state_value_fire * (1 - done_tensor)
        fire_advantage = fire_target - state_value_fire
        
        water_target = water_reward_tensor + GAMMA * next_state_value_water * (1 - done_tensor)
        water_advantage = water_target - state_value_water
        
        # Calculate losses
        fire_action_dist = torch.distributions.Categorical(action_probs_fire)
        fire_action_log_probs = fire_action_dist.log_prob(torch.tensor(fire_action))
        
        water_action_dist = torch.distributions.Categorical(action_probs_water)
        water_action_log_probs = water_action_dist.log_prob(torch.tensor(water_action))
        
        # Actor losses (policy gradient)
        fire_actor_loss = -fire_action_log_probs * fire_advantage.detach()
        water_actor_loss = -water_action_log_probs * water_advantage.detach()
        
        # Critic losses (value function)
        fire_critic_loss = F.mse_loss(state_value_fire, fire_target.detach())
        water_critic_loss = F.mse_loss(state_value_water, water_target.detach())
        
        # Entropy to encourage exploration
        fire_entropy = -torch.sum(action_probs_fire * torch.log(action_probs_fire + 1e-8))
        water_entropy = -torch.sum(action_probs_water * torch.log(action_probs_water + 1e-8))
        
        # Additional loss term to encourage similar actions when agents are close
        position_fire = torch.tensor(state[-6:-4], dtype=torch.float32)
        position_water = torch.tensor(state[-4:-2], dtype=torch.float32)
        distance = torch.norm(position_fire - position_water)
        
        # When agents are close, encourage similar actions
        action_similarity = torch.abs(action_probs_fire - action_probs_water).sum()
        proximity_loss = torch.where(distance < 3.0, action_similarity, torch.tensor(0.0))
        
        # Total loss - including joint loss components to encourage cooperation
        total_loss = (
            fire_actor_loss + water_actor_loss +
            0.5 * (fire_critic_loss + water_critic_loss) -
            0.01 * (fire_entropy + water_entropy) +
            0.1 * proximity_loss  # Add proximity-based coordination loss
        )
        
        # Update network
        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()

# DQN Agent (unchanged)
class DQNAgent:
    def __init__(self, input_dim, action_dim):
        self.input_dim = input_dim
        self.action_dim = action_dim
        self.policy_net = DQN(input_dim, action_dim)
        self.target_net = DQN(input_dim, action_dim)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_update_freq = 100
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=0.001)
        self.memory = ReplayMemory(10000)
        self.batch_size = 64
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.gamma = GAMMA
        self.steps_done = 0
        
    def select_action(self, state, explore=True):
        if explore and random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
                return self.policy_net(state_tensor).max(1)[1].item()
    
    def update_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    def update_target_network(self):
        if self.steps_done % self.target_update_freq == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def push_to_memory(self, state, action, reward, next_state, done):
        self.memory.push(state, action, reward, next_state, done)
        self.steps_done += 1
    
    def learn(self):
        if len(self.memory) < self.batch_size:
            return
        
        experiences = self.memory.sample(self.batch_size)
        batch = Experience(*zip(*experiences))
        
        state_batch = torch.FloatTensor(batch.state)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1)
        reward_batch = torch.FloatTensor(batch.reward)
        next_state_batch = torch.FloatTensor(batch.next_state)
        done_batch = torch.FloatTensor(batch.done)
        
        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the columns of actions taken
        state_action_values = self.policy_net(state_batch).gather(1, action_batch)
        
        # Compute V(s_{t+1}) for all next states
        next_state_values = torch.zeros(self.batch_size)
        with torch.no_grad():
            next_state_values = self.target_net(next_state_batch).max(1)[0]
        
        # Compute the expected Q values
        expected_state_action_values = reward_batch + (self.gamma * next_state_values * (1 - done_batch))
        
        # Compute Huber loss
        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
        
        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

# Game Environment
class GameEnvironment:
    def __init__(self):
        self.reset()
        
        # Define state size: flattened grid plus agent positions
        self.state_size = (GRID_WIDTH * GRID_HEIGHT) + 6  # Grid + agent positions (y,x for each agent)
        self.action_size = NUM_ACTIONS
        
        # Initialize agents - use shared A2C for F & W
        self.shared_a2c_agent = SharedA2CAgent(self.state_size, self.action_size)
        self.dqn_agent_green = DQNAgent(self.state_size, self.action_size)
        
        # Track buttons pressed
        self.yellow_buttons_pressed = 0
        self.purple_buttons_pressed = 0
        self.total_buttons = 0  # Will be counted during reset
        
        # Track collaboration
        self.fire_water_cooperation = False
        
        # Track goals reached
        self.fire_reached_goal = False
        self.water_reached_goal = False
        
        # Track gems collected
        self.fire_gems = 0
        self.water_gems = 0
        self.total_fire_gems = 0
        self.total_water_gems = 0
    
    def reset(self):
        # Create grid
        self.grid = [["0" for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]
        
        # Initial agent positions
        self.agents = {
            "F": [18, 1],
            "W": [15, 1],
            "G": [2, 19]
        }
        
        # Store original grid state
        self.original_grid = None
        
        # Place agents on grid
        for agent, pos in self.agents.items():
            self.grid[pos[0]][pos[1]] = agent
        
        # Poison tiles
        for x in range(10, 13): self.grid[18][x] = "RP"
        for x in range(17, 20): self.grid[18][x] = "BP"
        for x in range(18, 21): self.grid[14][x] = "GP"
        
        # Gems
        self.grid[17][11] = self.grid[8][10] = "RB"
        self.grid[17][18] = self.grid[10][20] = "BB"
        
        # Count total gems
        self.total_fire_gems = sum(row.count("RB") for row in self.grid)
        self.total_water_gems = sum(row.count("BB") for row in self.grid)
        
        # Goals
        self.grid[2][26] = "FG"
        self.grid[2][28] = "WG"
        
        # Slides
        for x in range(1, 4): self.grid[9][x] = "YS"
        for x in range(26, 29): self.grid[6][x] = "PS"
        
        # Buttons
        self.grid[8][6] = self.grid[12][9] = "YB"
        self.grid[8][14] = self.grid[5][22] = "PB"
        
        # Count total buttons
        self.total_buttons = sum(row.count("YB") for row in self.grid) + sum(row.count("PB") for row in self.grid)
        
        # Platforms (walls)
        for i in range(30): self.grid[0][i] = self.grid[19][i] = "1"
        for i in range(20): self.grid[i][0] = self.grid[i][29] = "1"
        self.grid[1][1] = self.grid[1][2] = "1"
        for i in range(4, 29): self.grid[3][i] = "1"
        self.grid[4][1] = self.grid[4][2] = self.grid[5][1] = self.grid[5][2] = "1"
        for i in range(1, 26): self.grid[6][i] = "1"
        self.grid[7][18] = self.grid[7][19] = self.grid[7][20] = "1"
        self.grid[8][18] = self.grid[8][19] = self.grid[8][20] = "1"
        for i in range(4, 16): self.grid[9][i] = "1"
        for i in range(17, 22): self.grid[11][i] = "1"
        for i in range(22, 29): self.grid[10][i] = "1"
        self.grid[10][16] = "1"
        self.grid[11][26] = self.grid[11][27] = self.grid[11][28] = "1"
        self.grid[12][26] = self.grid[12][27] = self.grid[12][28] = "1"
        for i in range(1, 13): self.grid[13][i] = "1"
        self.grid[14][13] = "1"
        for i in range(14, 27): self.grid[15][i] = "1"
        for i in range(1, 7): self.grid[16][i] = "1"
        self.grid[17][27] = self.grid[17][28] = self.grid[18][27] = self.grid[18][28] = "1"
        
        # Store original grid after setup (for resetting after collisions)
        self.original_grid = [row[:] for row in self.grid]
        
        # Reset game state
        self.yellow_buttons_pressed = 0
        self.purple_buttons_pressed = 0
        self.yellow_button_pressed_by = None  # Added initialization
        self.purple_button_pressed_by = None  # Added initialization
        self.fire_water_cooperation = False
        self.fire_reached_goal = False
        self.water_reached_goal = False
        self.fire_gems = 0
        self.water_gems = 0
        self.fire_poison = False  # Added initialization
        self.water_poison = False  # Added initialization
        self.fire_gem = False  # Added initialization
        self.water_gem = False  # Added initialization
        self.done = False
        
        return self.get_state()
    
    def get_state(self):
        # Flattened grid representation (one-hot encoded)
        # Define mapping for one-hot encoding
        tile_types = ["0", "1", "F", "W", "G", "RP", "BP", "GP", "FG", "WG", "RB", "BB", "YS", "PS", "YB", "PB"]
        tile_to_idx = {tile: idx for idx, tile in enumerate(tile_types)}
        # For simplicity, we'll use a simpler representation here
        grid_flat = []
        for row in self.grid:
            for cell in row:
                if cell == "0":
                    grid_flat.append(0)  # Empty
                elif cell == "1":
                    grid_flat.append(1)  # Wall
                elif cell == "F":
                    grid_flat.append(2)  # Fireboy
                elif cell == "W":
                    grid_flat.append(3)  # Watergirl
                elif cell == "G":
                    grid_flat.append(4)  # Green agent
                elif cell == "RP":
                    grid_flat.append(5)  # Red poison
                elif cell == "BP":
                    grid_flat.append(6)  # Blue poison
                elif cell == "GP":
                    grid_flat.append(7)  # Green poison
                elif cell == "FG":
                    grid_flat.append(8)  # Fireboy goal
                elif cell == "WG":
                    grid_flat.append(9)  # Watergirl goal
                elif cell == "RB":
                    grid_flat.append(10)  # Red gem
                elif cell == "BB":
                    grid_flat.append(11)  # Blue gem
                elif cell == "YS":
                    grid_flat.append(12)  # Yellow slide
                elif cell == "PS":
                    grid_flat.append(13)  # Purple slide
                elif cell == "YB":
                    grid_flat.append(14)  # Yellow button
                elif cell == "PB":
                    grid_flat.append(15)  # Purple button
                else:
                    grid_flat.append(0)  # Default to empty
                    
        # Add agent positions
        for agent, pos in self.agents.items():
            grid_flat.extend(pos)
            
        return grid_flat
    
    def is_valid_move(self, agent, y, x):
        # Check bounds
        if not (0 <= x < GRID_WIDTH and 0 <= y < GRID_HEIGHT):
            return False
        
        # Check if the cell is a wall
        if self.grid[y][x] == "1":
            return False
        
        # Check if the cell contains another agent
        if self.grid[y][x] in ["F", "W", "G"] and self.grid[y][x] != agent:
            return False
            
        return True

    def check_special_interaction(self, agent, y, x):
        cell = self.grid[y][x]
        rewards = {"F": 0, "W": 0, "G": 0}
        game_over = False
        
        # Poison interaction - preserve poison tiles
        if agent == "F" and cell == "BP":
            rewards["F"] = -10  # Fireboy dies in red poison
            rewards["W"] = -5
            rewards["G"] = 5    # Green agent gets rewarded when opponents fail
            game_over = True
        elif agent == "F" and cell == "RP":
            rewards["F"] = -5   # Fireboy gets penalty for red poison
            rewards["W"] = 0
            rewards["G"] = 2    # Small reward for green agent
            self.fire_poison = True
        elif agent == "W" and cell == "RP":
            rewards["W"] = -10  # Watergirl dies in blue poison
            rewards["F"] = -5
            rewards["G"] = 5    # Green agent gets rewarded when opponents fail
            game_over = True
        elif agent == "W" and cell == "BP":
            rewards["W"] = -5   # Watergirl gets penalty for blue poison
            rewards["F"] = 0
            rewards["G"] = 2    # Small reward for green agent
            self.water_poison = True
        elif agent in ["F", "W"] and cell == "GP":
            rewards[agent] = -10  # Both die in green poison
            rewards["G"] = -10    # Green agent also penalized for green poison
            game_over = True
        elif agent == "G" and cell in ["RP", "BP", "GP"]:
            rewards["G"] = -5     # Green agent penalized for touching any poison
            game_over = False
        
        # Gem collection
        if agent == "F" and cell == "RB":
            rewards["F"] = 5    # Fireboy collects red gem
            rewards["W"] = 1
            rewards["G"] = -2   # Green agent penalized when opponents succeed
            self.fire_gems += 1
            self.grid[y][x] = "0"  # Remove the gem (gems should be collected)
        elif agent == "W" and cell == "BB":
            rewards["W"] = 5    # Watergirl collects blue gem
            rewards["F"] = 1
            rewards["G"] = -2   # Green agent penalized when opponents succeed
            self.water_gems += 1
            self.grid[y][x] = "0"  # Remove the gem (gems should be collected)
        elif agent == "G" and cell in ["RB", "BB"]:
            rewards["G"] = 5    # Green agent collects gems
            rewards["F"] = -1   # Small penalty for F&W when green collects gems
            rewards["W"] = -1
            self.grid[y][x] = "0"  # Remove the gem

        # Wrong gem collection penalties
        elif agent == "F" and cell == "BB":
            rewards["F"] = -2    # Penalty for Fireboy trying to collect blue gem
            rewards["W"] = -1    # Small penalty for Watergirl
            rewards["G"] = 2     # Green agent gets rewarded when opponents fail
            self.water_gem = True
        elif agent == "W" and cell == "RB":
            rewards["W"] = -2    # Penalty for Watergirl trying to collect red gem
            rewards["F"] = -1    # Small penalty for Fireboy
            rewards["G"] = 2     # Green agent gets rewarded when opponents fail
            self.fire_gem = True
        
        # Goal reached - goals remain in place
        if agent == "F" and cell == "FG":
            rewards["F"] = 10    # Fireboy reaches goal
            rewards["W"] = 3
            rewards["G"] = -10   # Green agent gets penalty
            self.fire_reached_goal = True
            # Goal remains in place
        elif agent == "W" and cell == "WG":
            rewards["W"] = 10    # Watergirl reaches goal
            rewards["F"] = 3
            rewards["G"] = -10   # Green agent gets penalty
            self.water_reached_goal = True
            # Goal remains in place
        elif agent == "G" and cell in ["FG", "WG"]:
            rewards["G"] = -5    # Green agent penalized for touching goals
            rewards["F"] = 0
            rewards["W"] = 0
            
        # Button pressing - buttons remain in place
        if agent in ["F", "W", "G"] and cell == "YB":
            # Yellow button activation
            if agent == "G":
                rewards["G"] = 3
                rewards["F"] = -1
                rewards["W"] = -1
            else:
                # Check if both F&W agents are near each other (cooperation)
                f_pos = self.agents["F"]
                w_pos = self.agents["W"]
                if abs(f_pos[0] - w_pos[0]) + abs(f_pos[1] - w_pos[1]) <= 3:  # Manhattan distance <= 3
                    rewards["F"] = 8
                    rewards["W"] = 8
                    rewards["G"] = -5    # Greater penalty for green agent when F&W cooperate
                    self.fire_water_cooperation = True
                else:
                    rewards["F"] = -2 if agent == "F" else 0
                    rewards["W"] = -2 if agent == "W" else 0
                    rewards["G"] = 2     # Green agent rewarded when F&W fail to cooperate
            
            # Track which agent is on the button and open/close slides accordingly
            self.yellow_button_pressed_by = agent
            self.yellow_buttons_pressed += 1
            # Open yellow slides - will be handled in step method
                
        if agent in ["F", "W", "G"] and cell == "PB":
            # Purple button activation
            if agent == "G":
                rewards["G"] = 3
                rewards["F"] = -1
                rewards["W"] = -1
            else:
                # Check if both F&W agents are near each other (cooperation)
                f_pos = self.agents["F"]
                w_pos = self.agents["W"]
                if abs(f_pos[0] - w_pos[0]) + abs(f_pos[1] - w_pos[1]) <= 3:  # Manhattan distance <= 3
                    rewards["F"] = 8
                    rewards["W"] = 8
                    rewards["G"] = -5    # Greater penalty for green agent when F&W cooperate
                    self.fire_water_cooperation = True
                else:
                    rewards["F"] = -2 if agent == "F" else 0
                    rewards["W"] = -2 if agent == "W" else 0
                    rewards["G"] = 2     # Green agent rewarded when F&W fail to cooperate
                    
            # Track which agent is on the button and open/close slides accordingly
            self.purple_button_pressed_by = agent
            self.purple_buttons_pressed += 1
            # Open purple slides - will be handled in step method

        # Additional rewards for proximity to encourage cooperation
        f_pos = self.agents["F"]
        w_pos = self.agents["W"]
        proximity = abs(f_pos[0] - w_pos[0]) + abs(f_pos[1] - w_pos[1])
        if proximity <= 2 and agent in ["F", "W"]:
            rewards["F"] += 0.2
            rewards["W"] += 0.2
            rewards["G"] -= 0.1  # Small penalty for green when F&W are cooperating
        
        return rewards, game_over
    
    def step(self, fire_action, water_action, green_action):
        if self.done:
            return self.get_state(), {"F": 0, "W": 0, "G": 0}, self.done, {}
            
        rewards = {"F": 0, "W": 0, "G": 0}
        game_over = False
        
        # Reset button states at the beginning of each step
        previous_yellow_pressed = getattr(self, "yellow_button_pressed_by", None)
        previous_purple_pressed = getattr(self, "purple_button_pressed_by", None)
        self.yellow_button_pressed_by = None
        self.purple_button_pressed_by = None
        
        # Get grid dimensions
        height = len(self.grid)
        width = len(self.grid[0]) if height > 0 else 0
        
        # First pass: Handle and track slide states based on button states
        # Replace yellow slides with empty space when yellow button is pressed
        if previous_yellow_pressed is not None:
            for y in range(height):
                for x in range(width):
                    if self.grid[y][x] == "YS":
                        self.grid[y][x] = "0"  # Open yellow slides
        
        # Replace purple slides with empty space when purple button is pressed
        if previous_purple_pressed is not None:
            for y in range(height):
                for x in range(width):
                    if self.grid[y][x] == "PS":
                        self.grid[y][x] = "0"  # Open purple slides
        
        # Process agent movements
        action_map = {
            "F": fire_action,
            "W": water_action,
            "G": green_action
        }
        
        for agent, action_idx in action_map.items():
            # Get current position
            y, x = self.agents[agent]
            
            # Get movement based on action
            dy, dx = ACTIONS[action_idx]
            new_y, new_x = y + dy, x + dx
            
            # Check if move is valid
            if self.is_valid_move(agent, new_y, new_x):
                # Check for special interactions
                interaction_rewards, interaction_game_over = self.check_special_interaction(agent, new_y, new_x)
                
                # Add interaction rewards
                for key, value in interaction_rewards.items():
                    rewards[key] += value
                    
                # Update game over flag
                game_over = game_over or interaction_game_over
                
                # Move agent if there's no game over
                if not game_over:
                    # Clear current position
                    if self.grid[y][x] == agent:  # Only clear if the agent is still there
                        if self.original_grid and self.original_grid[y][x] in ["YB", "PB", "YS", "PS", "FG", "WG", "RP", "BP", "GP"]:
                            self.grid[y][x] = self.original_grid[y][x]  # Restore original tile
                        else:
                            self.grid[y][x] = "0"  # Set to empty
                    
                    # Move to new position
                    if self.grid[new_y][new_x] not in ["FG", "WG", "YB", "PB"]:  # Don't overwrite goals or buttons
                        self.grid[new_y][new_x] = agent
                    
                    # Update agent position
                    self.agents[agent] = [new_y, new_x]
            else:
                # Penalty for invalid move
                rewards[agent] -= 0.1
        
        # Small time penalty to encourage faster solutions
        for agent in rewards:
            rewards[agent] -= 0.01
        
        # Check if both Fireboy and Watergirl reached their goals
        if self.fire_reached_goal and self.water_reached_goal:
            rewards["F"] += 20
            rewards["W"] += 20
            rewards["G"] -= 10
            game_over = True
            print("Both agents reached their goals! Success!")
        
        # Bonus rewards for collecting all gems
        if self.fire_gems == self.total_fire_gems and self.water_gems == self.total_water_gems:
            rewards["F"] += 15
            rewards["W"] += 15
            rewards["G"] -= 5
            print("All gems collected! Bonus awarded!")
        
        # Bonus for pressing all buttons (encouraging complete exploration)
        if self.yellow_buttons_pressed + self.purple_buttons_pressed == self.total_buttons:
            rewards["F"] += 10
            rewards["W"] += 10
            rewards["G"] -= 3
            print("All buttons pressed! Bonus awarded!")
            
        # Update done flag
        self.done = game_over
        
        return self.get_state(), rewards, self.done, {}
    
    def render(self):
        if not RENDER:
            return
            
        screen.fill((0, 0, 0))
        
        for y in range(GRID_HEIGHT):
            for x in range(GRID_WIDTH):
                tile = self.grid[y][x]
                color = COLORS.get(tile, BASE_COLOR)

                # Agent hearts
                if tile in ["F", "W", "G"]:
                    pygame.draw.rect(screen, color, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                    pygame.draw.polygon(screen, color, [
                        (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + 6),
                        (x*TILE_SIZE + 6, y*TILE_SIZE + TILE_SIZE//2),
                        (x*TILE_SIZE + TILE_SIZE - 6, y*TILE_SIZE + TILE_SIZE//2),
                    ])
                    pygame.draw.circle(screen, color, (x*TILE_SIZE + TILE_SIZE//3, y*TILE_SIZE + TILE_SIZE//3), 5)
                    pygame.draw.circle(screen, color, (x*TILE_SIZE + 2*TILE_SIZE//3, y*TILE_SIZE + TILE_SIZE//3), 5)

                # Gems
                elif tile in ["RB", "BB"]:
                    pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                    pygame.draw.polygon(screen, color, [
                        (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE),
                        (x*TILE_SIZE + TILE_SIZE, y*TILE_SIZE + TILE_SIZE//2),
                        (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + TILE_SIZE),
                        (x*TILE_SIZE, y*TILE_SIZE + TILE_SIZE//2)
                    ])

                # Buttons
                elif tile in ["YB", "PB"]:
                    pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                    pygame.draw.circle(screen, color, (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + TILE_SIZE//2), TILE_SIZE//4)

                # Goals
                elif tile in ["FG", "WG"]:
                    pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                    pygame.draw.polygon(screen, color, [
                        (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE),
                        (x*TILE_SIZE, y*TILE_SIZE + TILE_SIZE),
                        (x*TILE_SIZE + TILE_SIZE, y*TILE_SIZE + TILE_SIZE)
                    ])

                else:
                    pygame.draw.rect(screen, color, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))

                pygame.draw.rect(screen, (160, 82, 45), (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)
                
        # Display stats
        font = pygame.font.SysFont('Arial', 14)
        
        # # Fire stats
        # fire_text = f"Fire: G:{self.fire_gems}/{self.total_fire_gems} Goal:{self.fire_reached_goal}"
        # fire_surf = font.render(fire_text, True, COLORS["F"])
        # screen.blit(fire_surf, (10, 10))
        
        # # Water stats
        # water_text = f"Water: G:{self.water_gems}/{self.total_water_gems} Goal:{self.water_reached_goal}"
        # water_surf = font.render(water_text, True, COLORS["W"])
        # screen.blit(water_surf, (10, 30))
        
        # # Button stats
        # button_text = f"Buttons: Y:{self.yellow_buttons_pressed} P:{self.purple_buttons_pressed} Coop:{self.fire_water_cooperation}"
        # button_surf = font.render(button_text, True, (255, 255, 255))
        # screen.blit(button_surf, (10, 50))
        
        pygame.display.flip()
        clock.tick(FPS)

# Main training loop
def train():
    env = GameEnvironment()
    episodes = 0
    best_reward = -float('inf')
    
    # Training statistics
    stats = {
        "episode_rewards": [],
        "fire_water_cooperation": [],
        "goals_reached": [],
        "gems_collected": []
    }
    
    try:
        while episodes < MAX_EPISODES:
            # Reset environment
            state = env.reset()
            total_rewards = {"F": 0, "W": 0, "G": 0}
            done = False
            
            while not done:
                # Select actions
                fire_action = env.shared_a2c_agent.select_action(state, "F")
                water_action = env.shared_a2c_agent.select_action(state, "W")
                green_action = env.dqn_agent_green.select_action(state)
                
                # Take action
                next_state, rewards, done, _ = env.step(fire_action, water_action, green_action)
                
                # Update DQN agent's memory
                env.dqn_agent_green.push_to_memory(state, green_action, rewards["G"], next_state, done)
                
                # Learn
                env.shared_a2c_agent.update(state, fire_action, water_action, rewards["F"], rewards["W"], next_state, done)
                env.dqn_agent_green.learn()
                
                # Update total rewards
                for agent, reward in rewards.items():
                    total_rewards[agent] += reward
                
                # Render
                env.render()
                
                # Update state
                state = next_state
            
            # Update DQN agent's epsilon
            env.dqn_agent_green.update_epsilon()
            env.dqn_agent_green.update_target_network()
            
            # Log statistics
            episodes += 1
            combined_reward = total_rewards["F"] + total_rewards["W"]
            stats["episode_rewards"].append(combined_reward)
            stats["fire_water_cooperation"].append(1 if env.fire_water_cooperation else 0)
            stats["goals_reached"].append(1 if env.fire_reached_goal and env.water_reached_goal else 0)
            stats["gems_collected"].append(env.fire_gems + env.water_gems)
            
            # Save best model if performance improved
            if combined_reward > best_reward:
                best_reward = combined_reward
                torch.save(env.shared_a2c_agent.network.state_dict(), "best_a2c_model.pth")
                torch.save(env.dqn_agent_green.policy_net.state_dict(), "best_dqn_model.pth")
                print(f"Episode {episodes}: New best model saved with reward {best_reward}")
            
            # Print statistics every 10 episodes
            if episodes % 10 == 0:
                print(f"Episode {episodes}/{MAX_EPISODES}")
                print(f"Fire Reward: {total_rewards['F']:.2f}, Water Reward: {total_rewards['W']:.2f}, Green Reward: {total_rewards['G']:.2f}")
                print(f"Goals Reached: Fire={env.fire_reached_goal}, Water={env.water_reached_goal}")
                print(f"Gems Collected: Fire={env.fire_gems}/{env.total_fire_gems}, Water={env.water_gems}/{env.total_water_gems}")
                print(f"Buttons Pressed: Yellow={env.yellow_buttons_pressed}, Purple={env.purple_buttons_pressed}")
                print(f"Cooperation: {env.fire_water_cooperation}")
                print("-" * 50)
    
    except KeyboardInterrupt:
        print("Training interrupted by user")
    
    finally:
        # Save final models
        torch.save(env.shared_a2c_agent.network.state_dict(), "final_a2c_model.pth")
        torch.save(env.dqn_agent_green.policy_net.state_dict(), "final_dqn_model.pth")
        
        # Plot training statistics
        try:
            import matplotlib.pyplot as plt
            
            # Rewards
            plt.figure(figsize=(12, 8))
            plt.subplot(2, 2, 1)
            plt.plot(stats["episode_rewards"])
            plt.title("Total F&W Rewards per Episode")
            plt.xlabel("Episode")
            plt.ylabel("Reward")
            
            # Cooperation
            plt.subplot(2, 2, 2)
            plt.plot(stats["fire_water_cooperation"])
            plt.title("F&W Cooperation per Episode")
            plt.xlabel("Episode")
            plt.ylabel("Cooperation (0/1)")
            
            # Goals reached
            plt.subplot(2, 2, 3)
            plt.plot(stats["goals_reached"])
            plt.title("Goals Reached per Episode")
            plt.xlabel("Episode")
            plt.ylabel("Both Goals Reached (0/1)")
            
            # Gems collected
            plt.subplot(2, 2, 4)
            plt.plot(stats["gems_collected"])
            plt.title("Total Gems Collected per Episode")
            plt.xlabel("Episode")
            plt.ylabel("Gems Collected")
            
            plt.tight_layout()
            plt.savefig("training_stats.png")
            print("Training statistics saved to training_stats.png")
        except ImportError:
            print("Matplotlib not available, skipping visualization")

# Main function
if __name__ == "__main__":
    train()

Episode 1: New best model saved with reward -41.97999999999995
